In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import glob
from colorthief import ColorThief
import colorsys


class LicensePlateDetector:
    def __init__(self, pth_weights: str, pth_cfg: str, pth_classes: str):
        self.net = cv2.dnn.readNet(pth_weights, pth_cfg)
        self.classes = []
        with open(pth_classes, 'r') as f:
            self.classes = f.read().splitlines()
        self.font = cv2.FONT_HERSHEY_PLAIN
        self.color = (255, 0, 0)
        self.coordinates = None
        self.img = None
        self.fig_image = None
        self.roi_image = None
        
        
    def detect(self, fig):
        
        self.img = fig
        img = fig.copy()
        height, width, _ = img.shape
        blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
        self.net.setInput(blob)
        output_layer_names = self.net.getUnconnectedOutLayersNames()
        layer_outputs = self.net.forward(output_layer_names)
        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores) 
                confidence = scores[class_id]
                if confidence > 0.2:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append((float(confidence)))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

        if len(indexes) > 0:
            for i in indexes.flatten():
                
                x, y, w, h = boxes[i]
                label = str(self.classes[class_ids[i]])
                confidence = str(round(confidences[i],2))
                cv2.rectangle(img, (x,y), (x + w, y + h), (0,255,0), 2)
                cv2.putText(img, label + ' ' + confidence, (x, y ), self.font, 1, (255, 255, 255), 3)
        self.fig_image = img
        self.coordinates = (x, y, w, h)
        
        return
    
    
    def crop_plate(self):
        
        x, y, w, h = self.coordinates
        roi = self.img[y:y + h, x:x + w]
        self.roi_image = roi
        
        return
    
    

lpd = LicensePlateDetector(
    pth_weights='yolov3-train_final.weights', 
    pth_cfg='yolov3-testing.cfg', 
    pth_classes='classes.txt'
)

# Detect license plate
path = glob.glob("samples-confidential/*.jpg")
count=1
for img in path:
    n = cv2.imread(img)
    lpd.detect(n)
    plt.figure(figsize=(26, 26))
    plt.imshow(cv2.cvtColor(lpd.fig_image, cv2.COLOR_BGR2RGB))
    plt.savefig('BoundedBoxOutput/'+str(count)+'.jpg')
    plt.show()
    lpd.crop_plate()
    plt.figure(figsize=(10, 4))
    plt.imshow(cv2.cvtColor(lpd.roi_image, cv2.COLOR_BGR2RGB))
    plt.savefig('NumberPlateCropped/'+str(count)+'.jpg')
    count=count+1




In [ ]:

def rgb_to_hsv(r, g, b):
  
        # R, G, B values are divided by 255
        # to change the range from 0..255 to 0..1:
        r, g, b = r / 255.0, g / 255.0, b / 255.0

        # h, s, v = hue, saturation, value
        cmax = max(r, g, b)    # maximum of r, g, b
        cmin = min(r, g, b)    # minimum of r, g, b
        diff = cmax-cmin       # diff of cmax and cmin.

        # if cmax and cmax are equal then h = 0
        if cmax == cmin: 
            h = 0

        # if cmax equal r then compute h
        elif cmax == r: 
            h = (60 * ((g - b) / diff) + 360) % 360

        # if cmax equal g then compute h
        elif cmax == g:
            h = (60 * ((b - r) / diff) + 120) % 360

        # if cmax equal b then compute h
        elif cmax == b:
            h = (60 * ((r - g) / diff) + 240) % 360

        # if cmax equal zero
        if cmax == 0:
            s = 0
        else:
            s = (diff / cmax) * 100

        # compute v
        v = cmax * 100
        return h, s, v
    
#Detect if dominant color of number plate is yellow
count=0
path = glob.glob("NumberPlateCropped/*.jpg")
for i in range(len(path)):
    n=path[i]

    color_thief = ColorThief(n)
    dominant_color = color_thief.get_color(quality=1)
    dominant_color=list(dominant_color)
    

    r=dominant_color[0]
    g=dominant_color[1]
    b=dominant_color[2]

    hsv=list(rgb_to_hsv(r,g,b))
    
    for a in range(len(hsv)):
        hsv[a]=int(hsv[a])

    yellow_start=list((50,40,40))
    yellow_end=list((60,100,100))

    a=all(hsv[i]>yellow_start[i] for i in range(len(hsv)))
    b=all(hsv[i]<=yellow_end[i] for i in range(len(hsv)))                                            
    if a and b:
        count=count+1
    

print("The number of yellow plates identified "+ ' '+str(count))



In [ ]:

def rgb_to_hsv(r, g, b):
  
        # R, G, B values are divided by 255
        # to change the range from 0..255 to 0..1:
        r, g, b = r / 255.0, g / 255.0, b / 255.0

        # h, s, v = hue, saturation, value
        cmax = max(r, g, b)    # maximum of r, g, b
        cmin = min(r, g, b)    # minimum of r, g, b
        diff = cmax-cmin       # diff of cmax and cmin.

        # if cmax and cmax are equal then h = 0
        if cmax == cmin: 
            h = 0

        # if cmax equal r then compute h
        elif cmax == r: 
            h = (60 * ((g - b) / diff) + 360) % 360

        # if cmax equal g then compute h
        elif cmax == g:
            h = (60 * ((b - r) / diff) + 120) % 360

        # if cmax equal b then compute h
        elif cmax == b:
            h = (60 * ((r - g) / diff) + 240) % 360

        # if cmax equal zero
        if cmax == 0:
            s = 0
        else:
            s = (diff / cmax) * 100

        # compute v
        v = cmax * 100
        return h, s, v
    
#Detect if dominant color of number plate is yellow
count=0


color_thief = ColorThief('NumberPlateCropped/131.jpg')
dominant_color = color_thief.get_color(quality=1)
dominant_color=list(dominant_color)
print(dominant_color)
r=dominant_color[0]
g=dominant_color[1]
b=dominant_color[2]

hsv=list(rgb_to_hsv(r,g,b))
print(hsv)
for a in range(len(hsv)):
    hsv[a]=int(hsv[a])

yellow_start=list((60,25,40))
yellow_end=list((60,100,100))

a=all(hsv[i]>yellow_start[i] for i in range(len(hsv)))
b=all(hsv[i]<=yellow_end[i] for i in range(len(hsv)))                                            
if a and b:
    count=count+1
    

print("The number of yellow plates identified "+ ' '+str(count))

